In [1]:
import sys, os
import pathlib

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import pathlib
from copy import deepcopy


In [2]:
ROOT_DIR = pathlib.Path().resolve().parent
print(ROOT_DIR)
sys.path.append(str(ROOT_DIR))
FIG_DIR = ROOT_DIR / 'results' / 'figures'
os.makedirs(FIG_DIR, exist_ok=True)
ERROR_ARR_SAVE_DIR = ROOT_DIR / 'results' / 'error_array'
os.makedirs(ERROR_ARR_SAVE_DIR, exist_ok=True)

/Users/1nao/GitHub/Gradient_Boosting_Meta_Tree


In [3]:
from gradient_boosting_meta_tree import metatree, normal
from sklearn.ensemble import GradientBoostingRegressor

In [4]:
SEED = 0
rng = np.random.default_rng(SEED)
np.random.seed(SEED)
MODEL_SEED = 0 # Seed for the learnmodel

# ベンチマークデータ読み込み

In [5]:
DIR_DATA = ROOT_DIR / 'data' / 'regression' / 'preprocessed_data'

In [6]:
# フォルダ名一覧から全データ名を取得
DATA_NAMES = os.listdir(DIR_DATA)
DATA_NAMES = np.sort([d for d in DATA_NAMES if d != '.DS_Store']).tolist()
DATA_NAMES

['Mpg',
 'abalone',
 'automobile',
 'cpu',
 'liver',
 'servo',
 'student',
 'wine_quality']

## データの読み込み

## 関数の定義

In [7]:
def load_data(
        data_name: str,
        dir_data: str=DIR_DATA,
        flag_lr: bool = False,
):
    # 個人の環境でpathを変更
    X_continuous_load = np.load(os.path.join(dir_data,data_name,'x_continuous.npy'))
    X_categorical = np.load(os.path.join(dir_data,data_name,'x_categorical.npy')).astype(int)
    y = np.load(os.path.join(dir_data,data_name,'y.npy'))

    # 連続説明変数の標準化
    mean = np.mean(X_continuous_load,axis=0)
    std = np.std(X_continuous_load,axis=0)
    X_continuous = (X_continuous_load - mean) / std

    # 線形回帰で定数項を入れる処理
    if flag_lr:
        tmp = np.copy(X_continuous)
        X_continuous = np.ones([tmp.shape[0],tmp.shape[1]+1])
        X_continuous[:,:-1] = tmp

    #標準化の処理
    y = (y - y.mean()) / y.std()

    return X_continuous, X_categorical, y

In [8]:
# すべてのデータのサンプル数と説明変数の数を確認
for data_name in DATA_NAMES:
    x_continuous,x_categorical,y = load_data(data_name,DIR_DATA)
    print(data_name, x_continuous.shape[0], x_continuous.shape[1] + x_categorical.shape[1])

Mpg 392 11
abalone 4177 10
automobile 159 57
cpu 209 36
liver 345 5
servo 167 12
student 395 43
wine_quality 4898 11


## 学習と指標計算を行う関数作成

In [9]:
# 関数の定義順の都合上木の数をここで指定
num_tree = 100

In [ ]:
from sklearn.metrics import mean_squared_error

In [11]:
sub_h_params_learnmodel = {
    'h0_tau_x': 1e1,
    'h0_m': 0.,
    'h0_tau': 1e0,
    'known_precision': True,
}
max_depth = 3 # change here in 3,5,15 and rerun to have each result
h0_split = 0.8
learning_rate = 0.5

In [12]:
MTGB_model_dict = {
    'model': metatree.SumOfMetaTreeLearnModel,
    'model_type': 'metatree',
    'init_params': {
        'c_num_metatrees': num_tree,
        'SubModel': normal,
        'sub_h0_params': sub_h_params_learnmodel,
        'c_max_depth': max_depth,
        'h0_split': h0_split,
        'learning_rate': learning_rate,
        'seed': MODEL_SEED,
    },
    'build_params': {
        'split_strategy': 'best',
        'building_scheme': 'depth_first',
        'calc_residual': True,
    }
}

## Define functions

In [13]:
def pred_measure_performance_gb(
        x_continuous_train, x_categorical_train, y_train,
        x_continuous_test, x_categorical_test, y_test,
        num_tree,
):
    x_train = np.concatenate([x_continuous_train, x_categorical_train], axis=1)
    x_test = np.concatenate([x_continuous_test, x_categorical_test], axis=1)

    gb = GradientBoostingRegressor(n_estimators=num_tree, learning_rate=learning_rate, max_depth=max_depth, random_state=MODEL_SEED)
    gb.fit(x_train, y_train)
    hat_y_train = gb.predict(x_train)
    hat_y_test = gb.predict(x_test)

    train_mse = mean_squared_error(y_train, hat_y_train)
    test_mse = mean_squared_error(y_test, hat_y_test)
    return train_mse, test_mse

In [14]:
def pred_measure_performance_mtgb(
        x_continuous_train, x_categorical_train, y_train,
        x_continuous_test, x_categorical_test, y_test,
        num_tree,
):
        data_info = {
                'c_dim_continuous': x_continuous_train.shape[1],
                'c_dim_categorical': x_categorical_train.shape[1],
        }
        init_inputs = {
                **MTGB_model_dict['init_params'],
                **data_info,
        }

        build_inputs = {
                **MTGB_model_dict['build_params'],
                'x_continuous_vecs': x_continuous_train,
                'x_categorical_vecs': x_categorical_train,
                'y_vec': y_train,
        }
        model = MTGB_model_dict['model'](**init_inputs)
        model.build_metatrees(**build_inputs)
        # model.calc_pred_dist()

        hat_y_train = model.make_prediction(
                x_continuous_train,
                x_categorical_train,
                loss='squared',
        )
        hat_y_test = model.make_prediction(
                x_continuous_test,
                x_categorical_test,
                loss='squared',
        )

        train_mse = mean_squared_error(y_train, hat_y_train)
        test_mse = mean_squared_error(y_test, hat_y_test)
        
        root_split_proba_list = [None for _ in model.hn_metatree_list]
        for i, metatree in enumerate(model.hn_metatree_list):
                root_split_proba_list[i] = metatree.root_node.hn_split

        return train_mse, test_mse, root_split_proba_list

In [ ]:
from sklearn.model_selection import KFold
def roop_kfold(
        data_name,
        num_tree,
        modelname,
        # do_metatree,
        visualize,
        num_cv=3,
        num_folds=5,
):
        num_trials = num_cv * num_folds
        x_continuous,x_categorical,y = load_data(data_name,DIR_DATA)
        
        # load mse list from ERROR_ARR_SAVE_DIR
        # try:
        #         train_mse_list = list(np.load(ERROR_ARR_SAVE_DIR / f'{data_name}_{modelname}_train_mse.npy'))
        #         test_mse_list = list(np.load(ERROR_ARR_SAVE_DIR / f'{data_name}_{modelname}_test_mse.npy'))
        # except:
        #         # tqdm.write(f'No file found. Create new list.')
        #         train_mse_list = []
        #         test_mse_list = []
        train_mse_list = []
        test_mse_list = []
        if modelname in ['MTGB']:
                root_split_proba_list_list = []

        with tqdm(total=num_cv, desc=f'{num_cv}CV', leave=False) as bar1:
                for i in range(num_cv):
                        with tqdm(total=num_folds, desc=f'{num_folds} folds', leave=False) as bar2:
                                kf = KFold(n_splits=num_folds, shuffle=True, random_state=SEED+i)
                                for j, (train_index, test_index) in enumerate(kf.split(y)):
                                        x_continuous_train, x_continuous_test = x_continuous[train_index], x_continuous[test_index]
                                        x_categorical_train, x_categorical_test = x_categorical[train_index], x_categorical[test_index]
                                        y_train, y_test = y[train_index], y[test_index]
                                        if  modelname == 'GB':
                                                train_mse, test_mse = pred_measure_performance_gb(
                                                        x_continuous_train, x_categorical_train, y_train,
                                                        x_continuous_test, x_categorical_test, y_test,
                                                        num_tree=num_tree,
                                                )
                                                train_mse_list.append(train_mse)
                                                test_mse_list.append(test_mse)
                                                bar2.update(1)
                                        elif modelname == 'MTGB':
                                                train_mse, test_mse, root_split_proba_list = pred_measure_performance_mtgb(
                                                        x_continuous_train, x_categorical_train, y_train,
                                                        x_continuous_test, x_categorical_test, y_test,
                                                        num_tree=num_tree,
                                                )
                                                train_mse_list.append(train_mse)
                                                test_mse_list.append(test_mse)
                                                root_split_proba_list_list.append(root_split_proba_list)
                                                bar2.update(1)
                                        else:
                                                raise ValueError('modelname is invalid')
                        bar1.update(1)
        np.save(ERROR_ARR_SAVE_DIR / f'{data_name}_{modelname}{num_tree}_depth{max_depth}_train_mse.npy', np.array(train_mse_list))
        np.save(ERROR_ARR_SAVE_DIR / f'{data_name}_{modelname}{num_tree}_depth{max_depth}_test_mse.npy', np.array(test_mse_list))
        if modelname == 'MTGB':
                np.save(ERROR_ARR_SAVE_DIR / f'{data_name}_{modelname}{num_tree}_depth{max_depth}_root_split_proba_list.npy', np.array(root_split_proba_list_list))
        return

## execute roops

In [16]:
MODELNAME_LIST = [
    'MTGB',
    'GB',
    ]

In [17]:
def experiment(
    data_name,
    num_tree,
    visualize,
    num_cv,
    num_folds,
):
    with tqdm(total=len(MODELNAME_LIST), desc="Model", leave=False) as pbar:
        for modelname in MODELNAME_LIST:
            txt = f'{data_name}: model = {modelname}'
            pbar.set_postfix_str(txt)
            roop_kfold(
                data_name,
                num_tree,
                modelname,
                visualize,
                num_cv,
                num_folds,
            )
            pbar.update(1)
    return

In [18]:
DATA_NAMES

['Mpg',
 'abalone',
 'automobile',
 'cpu',
 'liver',
 'servo',
 'student',
 'wine_quality']

In [19]:
num_cv = 3
num_folds = 5
visualize = False

df_resume = pd.DataFrame()
tqdm.write(f'num_cv: {num_cv}, num_folds: {num_folds}, visualize: {visualize}, num_tree: {num_tree}')
with tqdm(total=len(DATA_NAMES), desc="Data", leave=False) as pbar:
    for data_name in DATA_NAMES:
        x_continuous,x_categorical,y = load_data(data_name,DIR_DATA)
        txt = f'{data_name}: {x_continuous.shape[0]} samples, {x_continuous.shape[1]+x_categorical.shape[1]} features'
        pbar.set_postfix_str(txt)
        res_dict = experiment(
            data_name,
            num_tree,
            visualize,
            num_cv,
            num_folds,
        )
        pbar.update(1)

num_cv: 3, num_folds: 5, visualize: False, num_tree: 100


Data:   0%|          | 0/8 [00:00<?, ?it/s]

Model:   0%|          | 0/2 [00:00<?, ?it/s]

3CV:   0%|          | 0/3 [00:00<?, ?it/s]

5 folds:   0%|          | 0/5 [00:00<?, ?it/s]

5 folds:   0%|          | 0/5 [00:00<?, ?it/s]

5 folds:   0%|          | 0/5 [00:00<?, ?it/s]

KeyboardInterrupt: 